Wandb에 로그인하여 개인 api key를 넣어주세요 -> 체크포인트는 자동으로 매 에폭마다 완디비로 올라갑니다. -> 학습 중간에 끊기면 완디비에서 artifact를 다운받아 이어서 하면 됩니다. -> 동영상 평가는 영상을 업로드하고 input에 경로를 지정하면 output 동영상이 만들어집니다. -> output 영상을 다운받으면 됩니다.

** 가끔 AttributeError: Can't get attribute 'DFLoss' on.. 오류 뜨는데 ultralytics 버전이 달라서 그런겁니다.

1. import
1. 데이터
1. 모델 학습
1. 모델 중단시 학습 재개\
1. 모델 평가
1. 모델 영상 평가

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# 이미지 디렉토리와 라벨 디렉토리 경로 설정
image_dir = '/content/drive/MyDrive/배경'
label_dir = '/content/drive/MyDrive/배경 라벨 폴더'

# 이미지 파일 목록 가져오기
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

for image_file in image_files:
    # 라벨 파일 경로 설정
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_path = os.path.join(label_dir, label_file)

    # 빈 라벨 파일 생성
    with open(label_path, 'w') as f:
        pass

print("빈 라벨 파일 생성 완료.")


빈 라벨 파일 생성 완료.


In [ ]:
# 1. import

!pip install ultralytics
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# !pip install --upgrade ultralytics==8.0.186 wandb
!pip install wandb
!pip install torch torchvision torchaudio
!pip install opencv-python numpy matplotlib Pillow

import os
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os
import matplotlib.pyplot as plt

import shutil
%matplotlib inline
from sklearn.model_selection import train_test_split
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torch.optim as optim
from ultralytics import YOLO
from google.colab import drive
import wandb
wandb.login()
# drive.mount('/content/drive')
%cd /content

In [ ]:
# 2. 데이터

!curl  -L "https://app.roboflow.com/ds/28liIAkWiN?key=6Qrz2eATSn" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip    #640 이미지

!mv /content/train /content/dataset

images_list = os.listdir('/content/dataset/images')
labels_list = os.listdir('/content/dataset/labels')
os.mkdir('/content/dataset/images/train')
os.mkdir('/content/dataset/labels/train')
os.mkdir('/content/dataset/images/val')
os.mkdir('/content/dataset/labels/val')
os.mkdir('/content/dataset/images/test')
os.mkdir('/content/dataset/labels/test')
X_train, X_test, y_train, y_test = train_test_split(images_list, labels_list, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

for file in X_train:
    shutil.move(os.path.join('/content/dataset/images', file), os.path.join('/content/dataset/images/train', file))
    shutil.move(os.path.join('/content/dataset/labels', file.replace('.jpg', '.txt')), os.path.join('/content/dataset/labels/train', file.replace('.jpg', '.txt')))

for file in X_test:
    shutil.move(os.path.join('/content/dataset/images', file), os.path.join('/content/dataset/images/test', file))
    shutil.move(os.path.join('/content/dataset/labels', file.replace('.jpg', '.txt')), os.path.join('/content/dataset/labels/test', file.replace('.jpg', '.txt')))

for file in X_val:
    shutil.move(os.path.join('/content/dataset/images', file), os.path.join('/content/dataset/images/val', file))
    shutil.move(os.path.join('/content/dataset/labels', file.replace('.jpg', '.txt')), os.path.join('/content/dataset/labels/val', file.replace('.jpg', '.txt')))

del images_list,labels_list,X_train, X_test, y_train, y_test, X_val, y_val

with open('/content/dataset/data.yaml', 'w') as f:
    f.write('train: /content/dataset/images/train\n')
    f.write('val: /content/dataset/images/val\n')
    f.write('test: /content/dataset/images/test\n')
    f.write('nc: 4\n')
    f.write('names: [\'Bird\', \'Drone\', \'F.W.Drone\', \'Hellicopter\']\n')
    f.close()

In [ ]:
# 3.모델 학습시키기

from wandb.integration.ultralytics import add_wandb_callback

wandb.init(project="YOLOv8m바꾼 파라미터", job_type="training")

model = YOLO("yolov8m.pt")

add_wandb_callback(model, enable_model_checkpointing=True)

model.train(
    project="yolov8m",
    data="/content/dataset/data.yaml",
    epochs=100,
    imgsz=640,
    patience=15,
    batch=64,
    augment=False,
    cls = 3.0,
    lr0=0.001)

# model.val()

wandb.finish()


In [ ]:
# 4. 모델 학습 재개

import wandb
run = wandb.init()
artifact = run.use_artifact('suhyoun0106-hahahaha/YOLOv8nYHHHH/run_wnbt3643_model:v100', type='model')
artifact_dir = artifact.download()

In [ ]:
from wandb.integration.ultralytics import add_wandb_callback

model = YOLO("")

add_wandb_callback(model, enable_model_checkpointing=True)

model.train(
    project="yolov8m",
    data="/content/dataset/data.yaml",
    epochs=100,
    imgsz=640,
    patience=15,
    batch=64,
    augment=False,
    cls = 3.0
    lr0=0.001)

# model.val()

wandb.finish()


In [ ]:
# 5. 모델 평가
import wandb
run = wandb.init()
artifact = run.use_artifact('suhyoun0106-hahahaha/YOLOv8M 그레이 스케일링/run_h04celon_model:v54', type='model')
artifact_dir = artifact.download()

In [ ]:
model = YOLO('/content/artifacts/run_h04celon_model:v54/epoch54.pt')  # best.pt 파일 경로를 지정하세요

metrics = model.val(data='/content/dataset/data.yaml')     # 데이터셋 설정 파일 경로

print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")


In [ ]:
# 6. 모델 영상 평가

!pip install ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow

model = YOLO("/content/artifacts/run_h04celon_model:v54/epoch54.pt")


input_video_path = "/content/종합 test 영상.mp4"  # 비디오 파일 경로
output_video_path = "/content/output_video~~~.mp4"


cap = cv2.VideoCapture(input_video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    results = model.predict(frame, imgsz=640, conf=0.2, iou=0.9, agnostic_nms=True, augment=True)


    res_plotted = results[0].plot()


    out.write(res_plotted)

cap.release()
out.release()
cv2.destroyAllWindows()
